https://macappstore.org/poppler/

Press Command+Space and type Terminal and press enter/return key.
Copy and paste the following command in Terminal app:
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
and press enter/return key. Wait for the command to finish. If you are prompted to enter a password, please type your Mac user's login password and press ENTER. Mind you, as you type your password, it won't be visible on your Terminal (for security reasons), but rest assured it will work.
Now, copy/paste and run this command to make brew command available inside the Terminal: echo 'eval "$(/opt/homebrew/bin/brew shellenv)"' >> ~/.zprofile
Copy and paste the following command:
brew install poppler

echo 'export PATH=$PATH:/opt/homebrew/Cellar/poppler/25.02.0/bin' >> ~/.bash_profile
source ~/.bash_profile

restart terminal and jupter notes

copied the poppler folder to /usr/local/poppler too and added it to the path 
not sure which one worked /opt/homebrew/Cellar/ or /usr/local

In [1]:
from pdf2image import convert_from_path
filepath = "./data/Oct_24_Bills-_-Billing-and-Cost-Management-_-Global.pdf"
pages = convert_from_path(filepath, 500)

In [2]:
for count, page in enumerate(pages):
    page.save(f'./data/receipt1/receipt{count}.jpg', 'JPEG')

In [3]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip install groq

  Using cached groq-0.18.0-py3-none-any.whl (121 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached pydantic_core-2.27.2-cp310-cp310-macosx_11_0_arm64.whl (1.8 MB)

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [120]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.2/473.2 kB 7.2 MB/s eta 0:00:00a 0:00:01
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached jiter-0.8.2-cp310-cp310-macosx_11_0_arm64.whl (311 kB)

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [160]:
import base64
import groq
import os
from openai import OpenAI

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
      
def get_image_prompt(img_url: str, prev_response: str):
    base64_image = encode_image(img_url)
    prompt = [
            {
                
                "role": "user",
                "content":
                [
                    {
                        "type": "text",                         
                        # "text": f"You are tasked with calculating the bill for llama usage on different cloud deployments. You have to extract the relevant line items from the bill, simplify it and return a list of amounts. Please respond with a JSON object containing keys: 'Title', 'currency', 'explanation' (a brief explanation of your answer) and 'confidence'. For example: {{'amount': 100.98, 'currency': 'USD', 'explanation': 'From the given data the amount is ..', 'confidence': 0.98}}",
                        # "text": f"You are tasked with calculating the bill for llama usage on different cloud deployments. You have to extract all the information including summary and line items from the bill, and respond with a JSON object list containing the title, line item, amount and currency. Bill may have bill summary, line items, account details etc., structure JSON object appropriately. There may be multiple pages for the bill, the response for previous image of the bill was \n\n {prev_response}\n\n, make sure to respond in a way to append to the previous response, if previous response is null this is the first image in the receipt."
                         # "text": f"You are tasked with calculating the bill for llama usage on different cloud deployments. You have to extract most relevant details that add up to the grand total, include bill summary, account details and line items from the bill, and respond with a JSON object that is appropriate to the given bill, you may igonore the subline items which are not bolded. The bill have muliple pages which will be inputted in subsequent prompts, JSON response should contain 'LineItems'." #a sample format may be containing the following keys: Title, Account Details, Bill Summary, and Line Items. The Line Items key may contain a list of objects with Description, Usage Quantity, and Amount in USD keys. If there are multiple pages for the bill, please response with the same JSON format and key names, so it can be appended to the previous response. The response for the previous image of the bill was \n\n {prev_response}\n\n.", # Please structure your response as follows: {{'Title': 'Bill title', 'Account Details': {{'AccountName': {{'Description': 'Account description', 'UsageQuantity': quantity, 'Amount in USD': amount}}}}, 'BillSummary': {{'Total pre-tax': 'USD total', 'Amount in USD': 'USD amount'}}, 'LineItems': [{{'Description': 'Item description', 'Usage Quantity': quantity, 'Amount in USD': amount}}, ...]}}",
                        # "text": f"You are given a page of multiple page bill. You have to extract the line items that add up to the grand total of the bill, you may ignore the 0 amount line items. Please read the bill line by line and make sure you do not miss any non 0 entry and the amount has to be very accurate, return JSON response with line items 'LineItem' that should have title, description, amount and currency." # You have to extract most relevant details that add up to the grand total, include bill summary, account details and line items from the bill, and respond with a JSON object that is appropriate to the given bill, you may igonore the subline items which are not bolded. The bill have muliple pages which will be inputted in subsequent prompts, JSON response should contain 'LineItems'." #a sample format may be containing the following keys: Title, Account Details, Bill Summary, and Line Items. The Line Items key may contain a list of objects with Description, Usage Quantity, and Amount in USD keys. If there are multiple pages for the bill, please response with the same JSON format and key names, so it can be appended to the previous response. The response for the previous image of the bill was \n\n {prev_response}\n\n.", # Please structure your response as follows: {{'Title': 'Bill title', 'Account Details': {{'AccountName': {{'Description': 'Account description', 'UsageQuantity': quantity, 'Amount in USD': amount}}}}, 'BillSummary': {{'Total pre-tax': 'USD total', 'Amount in USD': 'USD amount'}}, 'LineItems': [{{'Description': 'Item description', 'Usage Quantity': quantity, 'Amount in USD': amount}}, ...]}}",
                        # "text": f"Extract and structure the data from the following multi-page receipt into a JSON format. Include all relevant financial details such as total amounts, payment information, service charges, regional expenses, and any other pertinent data. Ensure that the JSON output reflects the hierarchical structure of the receipt, capturing all key information across pages."
                        "text": f"Extract and structure the data from the following multi-page receipt into a JSON format. Ensure you understand the hierarchical structure of the receipt, the response should only contain the summary of each high level section. Json should be in the format {{'sections': [{{\"sectionName\": \"Name of the section\", \"description\": \"section description details\", \"amount\": 0.00, \"currency\": \"USD\", \"lineItems\": []}}]}}. lineItems may recursively contain sections depending on the heirarchy, the line item amounts should add up to the section amount. You can ignore the sections and lineitems with zero cost. A receipt image could be continuation of previous page and may not start with a section, The previous page response for your context is \n\n: {prev_response}." # Return response in a way it can be appended to prev response", # each section summary may for example have: {{'sectionName': 'Bill summary', 'description': 'Grand total', 'amount': '989.00', 'currency': 'USD', 'lineItems': []}}"
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    }, 
                ], 
            }
        ]
    return prompt
    
def get_llama_vision_response(prompt: list) -> str:
    client=groq.Groq(api_key=os.getenv('GROQ_API_KEY'))
    chat_completion = client.chat.completions.create(
        messages= prompt,
        model="llama-3.2-90b-vision-preview",
        response_format={"type": "json_object"},
        # temperature=0.2,
        # max_completion_tokens=MAX_TOKENS+1,
        # top_p=0.2,
        # response_model=StatementInfo
        # stream=False,
        # stop=None,
    )
    return chat_completion.choices[0].message.content

def get_openai_vision_response(prompt: list, retry: int = 0):
    try: 
        client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        chat_completion = client.chat.completions.create(
            messages= prompt,
            model="gpt-4o",
            response_format={"type": "json_object"},
            # max_tokens=MAX_TOKENS,
            temperature=0.1,
            top_p=0.1
        )
        return json.loads(chat_completion.choices[0].message.content)
    except:
        if retry == 0:
            time.sleep(3)
            return get_openai_vision_response(prompt, 1)
        return None

In [70]:
img_url = "./data/receipt1/receipt1.jpg"
prompt = get_image_prompt(img_url, json_response)

response = get_llama_vision_response(prompt)

In [26]:
response

'{\n   "Title" : "Charges by service",\n   "Account Details" : {\n      "Amazon Web Services, Inc." : {\n         "Description" : "BC Container Cloud",\n         "Usage Quantity" : 7488.75,\n         "Amount in USD" : 7488.75\n      }\n   },\n   "Bill Summary" : {\n      "Total pre-tax" : "USD 9,960.53",\n      "Amount in USD" : "USD 9,960.53"\n   },\n   "Line Items" : [\n      {\n         "Description" : "Amazon Elastic Compute Cloud NotOtherwise",\n         "Usage Quantity" : 0.001,\n         "Amount in USD" : 0.23\n      },\n      {\n         "Description" : "Amazon Elastic Compute Cloud NotOtherwise region  Pacific (Seoul)",\n         "Usage Quantity" : 0.001,\n         "Amount in USD" : 0.9\n      },\n      {\n         "Description" : "Amazon Elastic Compute Cloud NotOtherwise region  Pacific (Seoul)",\n         "Usage Quantity" : 1.93,\n         "Amount in USD" : 174.06\n      }\n   ]\n}'

In [27]:
json_response = json.loads(response)
json_response

{'Title': 'Charges by service',
 'Account Details': {'Amazon Web Services, Inc.': {'Description': 'BC Container Cloud',
   'Usage Quantity': 7488.75,
   'Amount in USD': 7488.75}},
 'Bill Summary': {'Total pre-tax': 'USD 9,960.53',
  'Amount in USD': 'USD 9,960.53'},
 'Line Items': [{'Description': 'Amazon Elastic Compute Cloud NotOtherwise',
   'Usage Quantity': 0.001,
   'Amount in USD': 0.23},
  {'Description': 'Amazon Elastic Compute Cloud NotOtherwise region  Pacific (Seoul)',
   'Usage Quantity': 0.001,
   'Amount in USD': 0.9},
  {'Description': 'Amazon Elastic Compute Cloud NotOtherwise region  Pacific (Seoul)',
   'Usage Quantity': 1.93,
   'Amount in USD': 174.06}]}

In [156]:
import time

def extract_number(filename):
    return int(''.join(filter(str.isdigit, filename)))

# load all images in the folder 
def get_image_filenames(folder_path: str):
    filenames = os.listdir(folder_path)
    sorted_filenames = sorted(filenames, key=extract_number)
    return sorted_filenames
    
# for each image extract response in json format 
# if its the first image prev response is 0 
# for other images -> append response to the previous rows 
def get_response(folder_path: str):
    prev_response = None
    complete_response = None
    files = get_image_filenames(folder_path)
    print(files)
    for file in files:
        img_url = os.path.join(folder_path, file)
        prompt = get_image_prompt(img_url, prev_response)
        response = get_openai_vision_response(prompt)
        print(response)
        print(f"\n****** {file} *****************************\n")
        # json_response = json.loads(response)
        prev_response = response
        # Initialize complete_response with the first json_response
        if complete_response is None:
            complete_response = response
        else:
            complete_response['sections'].extend(response.get('sections', []))
    return complete_response

In [161]:
folder_path = "./data/receipt1"
final_response = get_response(folder_path)

['receipt0.jpg', 'receipt1.jpg', 'receipt2.jpg', 'receipt3.jpg', 'receipt4.jpg', 'receipt5.jpg', 'receipt6.jpg', 'receipt7.jpg', 'receipt8.jpg', 'receipt9.jpg', 'receipt10.jpg', 'receipt11.jpg', 'receipt12.jpg', 'receipt13.jpg', 'receipt14.jpg', 'receipt15.jpg', 'receipt16.jpg', 'receipt17.jpg', 'receipt18.jpg', 'receipt19.jpg']
{'sections': [{'sectionName': 'AWS bill summary', 'description': 'Summary of charges for the billing period.', 'amount': 9960.53, 'currency': 'USD', 'lineItems': [{'sectionName': 'Amazon Web Services, Inc.', 'description': 'Charges for AWS services.', 'amount': 9960.53, 'currency': 'USD', 'lineItems': []}]}, {'sectionName': 'Highest cost by service provider', 'description': 'Details of the highest costs by service provider.', 'amount': 0.0, 'currency': 'USD', 'lineItems': [{'sectionName': 'Amazon Web Services, Inc.', 'description': 'Highest service and region spend details.', 'amount': 0.0, 'currency': 'USD', 'lineItems': [{'sectionName': 'Highest service spend

In [162]:
final_response

{'sections': [{'sectionName': 'AWS bill summary',
   'description': 'Summary of charges for the billing period.',
   'amount': 9960.53,
   'currency': 'USD',
   'lineItems': [{'sectionName': 'Amazon Web Services, Inc.',
     'description': 'Charges for AWS services.',
     'amount': 9960.53,
     'currency': 'USD',
     'lineItems': []}]},
  {'sectionName': 'Highest cost by service provider',
   'description': 'Details of the highest costs by service provider.',
   'amount': 0.0,
   'currency': 'USD',
   'lineItems': [{'sectionName': 'Amazon Web Services, Inc.',
     'description': 'Highest service and region spend details.',
     'amount': 0.0,
     'currency': 'USD',
     'lineItems': [{'sectionName': 'Highest service spend',
       'description': 'Elastic Compute Cloud',
       'amount': 7488.75,
       'currency': 'USD',
       'lineItems': []},
      {'sectionName': 'Highest AWS Region spend',
       'description': 'US West (Oregon)',
       'amount': 7833.87,
       'currency': '

In [163]:
def get_res_prompt(question: str, data: str):
    prompt = [
            {
                "role": "system",
                "content": "You are expert at Math and financial calculations. Output response in JSON format"
            },
            {
                "role": "user",
                "content":
                [
                    {
                        "type": "text",                         
                        # "text": f"You are tasked with calculating the bill for llama usage on different cloud deployments. You have to extract the relevant line items from the bill, simplify it and return a list of amounts. Please respond with a JSON object containing keys: 'Title', 'currency', 'explanation' (a brief explanation of your answer) and 'confidence'. For example: {{'amount': 100.98, 'currency': 'USD', 'explanation': 'From the given data the amount is ..', 'confidence': 0.98}}",
                        # "text": f"You are tasked with calculating the bill for llama usage on different cloud deployments. You have to extract all the information including summary and line items from the bill, and respond with a JSON object list containing the title, line item, amount and currency. Bill may have bill summary, line items, account details etc., structure JSON object appropriately. There may be multiple pages for the bill, the response for previous image of the bill was \n\n {prev_response}\n\n, make sure to respond in a way to append to the previous response, if previous response is null this is the first image in the receipt."
                         # "text": f"You are tasked with calculating the bill for llama usage on different cloud deployments. You have to extract most relevant details that add up to the grand total, include bill summary, account details and line items from the bill, and respond with a JSON object that is appropriate to the given bill, you may igonore the subline items which are not bolded. The bill have muliple pages which will be inputted in subsequent prompts, JSON response should contain 'LineItems'." #a sample format may be containing the following keys: Title, Account Details, Bill Summary, and Line Items. The Line Items key may contain a list of objects with Description, Usage Quantity, and Amount in USD keys. If there are multiple pages for the bill, please response with the same JSON format and key names, so it can be appended to the previous response. The response for the previous image of the bill was \n\n {prev_response}\n\n.", # Please structure your response as follows: {{'Title': 'Bill title', 'Account Details': {{'AccountName': {{'Description': 'Account description', 'UsageQuantity': quantity, 'Amount in USD': amount}}}}, 'BillSummary': {{'Total pre-tax': 'USD total', 'Amount in USD': 'USD amount'}}, 'LineItems': [{{'Description': 'Item description', 'Usage Quantity': quantity, 'Amount in USD': amount}}, ...]}}",
                        # "text": f"You are given a page of multiple page bill. You have to extract the line items that add up to the grand total of the bill, you may ignore the 0 amount line items. Please read the bill line by line and make sure you do not miss any non 0 entry and the amount has to be very accurate, return JSON response with line items 'LineItem' that should have title, description, amount and currency." # You have to extract most relevant details that add up to the grand total, include bill summary, account details and line items from the bill, and respond with a JSON object that is appropriate to the given bill, you may igonore the subline items which are not bolded. The bill have muliple pages which will be inputted in subsequent prompts, JSON response should contain 'LineItems'." #a sample format may be containing the following keys: Title, Account Details, Bill Summary, and Line Items. The Line Items key may contain a list of objects with Description, Usage Quantity, and Amount in USD keys. If there are multiple pages for the bill, please response with the same JSON format and key names, so it can be appended to the previous response. The response for the previous image of the bill was \n\n {prev_response}\n\n.", # Please structure your response as follows: {{'Title': 'Bill title', 'Account Details': {{'AccountName': {{'Description': 'Account description', 'UsageQuantity': quantity, 'Amount in USD': amount}}}}, 'BillSummary': {{'Total pre-tax': 'USD total', 'Amount in USD': 'USD amount'}}, 'LineItems': [{{'Description': 'Item description', 'Usage Quantity': quantity, 'Amount in USD': amount}}, ...]}}",
                        # "text": f"Extract and structure the data from the following multi-page receipt into a JSON format. Include all relevant financial details such as total amounts, payment information, service charges, regional expenses, and any other pertinent data. Ensure that the JSON output reflects the hierarchical structure of the receipt, capturing all key information across pages."
                        # "text": f"Extract and structure the data from the following multi-page receipt into a JSON format. Ensure you understand the hierarchical structure of the receipt, the response should only contain the summary of each high level section. Json should be in the format {{'sections': [{{\"sectionName\": \"Name of the section\", \"description\": \"section description details\", \"amount\": 0.00, \"currency\": \"USD\", \"lineItems\": []}}]}}. lineItems may recursively contain sections depending on the heirarchy, the line item amounts should add up to the section amount. You can ignore the sections and lineitems with zero cost. A receipt image could be continuation of previous page and may not start with a section, The previous page response for your context is \n\n: {prev_response}." # Return response in a way it can be appended to prev response", # each section summary may for example have: {{'sectionName': 'Bill summary', 'description': 'Grand total', 'amount': '989.00', 'currency': 'USD', 'lineItems': []}}"
                        "text": f"Based on the given json data extracted from a receipt, {question} \n\n {data}"
                    }
                ], 
            }
        ]
    return prompt



In [164]:
prompt = get_res_prompt("What is the total bill?", final_response)
response = get_openai_vision_response(prompt)
response

{'totalBill': 9960.53, 'currency': 'USD'}

In [165]:
prompt = get_res_prompt("What is the total bill for the services containing llama? Include explanation in response", final_response)
response = get_openai_vision_response(prompt)
response

{'totalLlamaBill': 0.01,
 'explanation': "The total bill for services containing 'llama' is derived from the 'Bedrock' section, specifically the line item 'Amazon Bedrock USW2-Llama2-3-90B-input-tokens', which has an amount of 0.01 USD. This is the only line item in the provided data that explicitly mentions 'llama'."}

In [166]:
prompt = get_res_prompt("What is the total discount? Include explanation in response", final_response)
response = get_openai_vision_response(prompt)
response

{'total_discount': 12841.2,
 'explanation': "The total discount is calculated from the 'Savings' section of the JSON data. The 'Savings' section lists various types of discounts applied, including 'Private Rate Card Discounts', 'Savings Plans Discounts', 'Enterprise Discount Program Discounts', and 'Usage Based Discounts'. The total amount of these discounts is 12841.2 USD, which represents the total discount applied to the AWS bill."}